In [ ]:
pip install gdown

In [ ]:
import gdown

url = 'https://drive.google.com/uc?export=download&id=1--W7LMimW3QF7x00CMxAUuUYcgDgSoFa'

output = 'file.npy'

gdown.download(url, output, quiet=False)

In [ ]:
import numpy as np

In [ ]:
with open('file.npy', 'rb') as f:
    X_train = np.load(f)
    X_val = np.load(f)
    Y_train = np.load(f)
    Y_val = np.load(f)
X_train.shape

In [ ]:
def get_category(level):
    categories = ["Healthy","Mild", "Moderate", "Severe", "Proliferate"]
    return categories[level]

In [ ]:
#checking data

from matplotlib import pyplot as plt
plt.imshow(X_train[7], interpolation='nearest')
plt.title(get_category(int(Y_train[7].sum(axis=0)) - 1))
plt.show()

In [ ]:
from tensorflow.keras.applications import VGG16
model = VGG16(weights='imagenet', include_top=False)
model.summary()

In [ ]:
vgg16_feature_list = []
from sklearn.utils import class_weight
#from keras.preprocessing import image
import cv2
for img in X_train:
    img = cv2.resize(img, (200, 200))
    img_data = np.expand_dims(img, axis=0)
    img_data = preprocess_input(img_data)
    vgg16_feature = model.predict(img_data)
    vgg16_feature_np = np.array(vgg16_feature)
    vgg16_feature_list.append(vgg16_feature_np.flatten())

train = np.array(vgg16_feature_list)

In [ ]:
train[0]

In [ ]:
train.shape

In [ ]:
vgg16_feature_list = []
for img in X_val:
    img = cv2.resize(img, (200, 200))
    img_data = np.expand_dims(img, axis=0)
    img_data = preprocess_input(img_data)
    vgg16_feature = model.predict(img_data)
    vgg16_feature_np = np.array(vgg16_feature)
    vgg16_feature_list.append(vgg16_feature_np.flatten())

test = np.array(vgg16_feature_list)

In [ ]:
test[0]

In [ ]:
test.shape

In [ ]:
y_train = Y_train.sum(axis=1) - 1
y_train[0]

In [ ]:
y_test = Y_val.sum(axis=1)-1
y_test[0]

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE
X_resampled, y_resampled = BorderlineSMOTE().fit_resample(train, y_train)

In [ ]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_resampled),
                                                 y_resampled)
print(class_weights)

> Class weights must be a dict parameter for LGBM Classifier not array

In [ ]:
class_weights={0:1,1:1,2:1,3:1,4:1}

In [ ]:
from lightgbm import LGBMClassifier
params = {
    'objective': 'multiclass',
    'num_class':5,
    'metric': 'multi_logloss',
    'class_weight':class_weights
}

classifier= LGBMClassifier(boosting_type='gbdt',objective='multiclass',num_class=5,metric='multi_logloss',class_weight=class_weights)
classifier.fit(X_resampled,y_resampled)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred_train=classifier.predict(X_resampled)
y_pred_test=classifier.predict(test)
print("Training Accuracy is:",accuracy_score(y_resampled,y_pred_train))
print("Testing Accuracy is:",accuracy_score(y_test,y_pred_test))

In [ ]:
from mlxtend.evaluate import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
cm=confusion_matrix(y_target=y_resampled,y_predicted=y_pred_train)
fig,ax=plot_confusion_matrix(conf_mat=cm,cmap='Blues')

plt.show()

In [ ]:
cm=confusion_matrix(y_target=y_test,y_predicted=y_pred_test)
fig,ax=plot_confusion_matrix(conf_mat=cm,cmap='Blues')

plt.show()